In [ ]:
from icevision.all import *
import numpy as np

import json
import os
from PIL import Image
from pycococreatortools import pycococreatortools

In [ ]:
model_type = models.torchvision.mask_rcnn
backbone = model_type.backbones.resnet34_fpn()

In [ ]:
mount_path = "/root/data/"

In [ ]:
parser = parsers.COCOMaskParser(annotations_filepath=f"{mount_path}/instances_slicks_test_v2_small.json", img_dir=f"{mount_path}/tiled_images_test_small_v2")

In [ ]:
train_records, valid_records = parser.parse()

In [ ]:
x=show_records(train_records[0:15], ncols=3, class_map=class_map)
plt.savefig("train_slick_examples.png")

In [ ]:
class_map = {
    "Infrastructure": 1,
    "Natural Seep": 2,
    "Coincident Vessel": 3,
    "Recent Vessel": 4,
    "Old Vessel": 5,
    "Ambiguous": 6,
    "Hard Negatives": 0,
}

class_map

Normalizing is best practice and necessary for icevision to properly display predicition results

In [ ]:
train_tfms = tfms.A.Adapter(
    [
        tfms.A.Normalize(),
    ]
)

In [ ]:
valid_tfms = tfms.A.Adapter([*tfms.A.resize_and_pad(size=512), tfms.A.Normalize()])

sourced from: https://airctic.com/0.8.1/getting_started_instance_segmentation/

In [ ]:
train_ds.records.autofix??

In [ ]:
train_ds = Dataset(train_records, train_tfms)
valid_ds = Dataset(valid_records, valid_tfms)

train_dl = model_type.train_dl(train_ds, batch_size=8, num_workers=6, shuffle=True) # adjust num_workers for your processor count
valid_dl = model_type.valid_dl(valid_ds, batch_size=8, num_workers=6, shuffle=False)

model = model_type.model(backbone=backbone(pretrained=False), num_classes=len(parser.class_map))

metrics = [COCOMetric(metric_type=COCOMetricType.mask, print_summary=False)]

learn = model_type.fastai.learner(dls=[train_dl, valid_dl], model=model, metrics=metrics)

lr = learn.lr_find()

In [ ]:
lr

The suggested learning rate makes getting to higher confidence predictions take too long. We picked the learning rate arbitrarily below to speed up getting to losses closer to .5 instead of greater than 1. 

In [ ]:
learn.fine_tune(30,2.511886486900039e-03)

a TODO is to debug the COCOMetric, it should not be -1 given that we are now acheiving detections that intersect with groundtruth.

In [ ]:
print(f"approximate time to train 30 epochs in minutes: {25*30/60}")


The predictions above .7 confidence that roughly line up with groundtruth demonstrates that icevision-trained models can produce predictions that look like they are headed in the correct direction, even for an imperfect training set.

In [ ]:
model_type.show_results??

In [ ]:
x = model_type.show_results(model, valid_ds, detection_threshold=.6)
plt.savefig("inference_results.png")

In [ ]:
show_results??